In [1]:
import requests
from requests_html import HTML
import pandas as pd

In [51]:
import time

In [2]:
url = "https://stackoverflow.com/questions/tagged/"
tag = "python"
query_filter = "Votes"
full_path = f"{url}{tag}?tab={query_filter}"
print(full_path)

https://stackoverflow.com/questions/tagged/python?tab=Votes


In [6]:
r = requests.get(full_path)
html_str = r.text
# print(r.text)

In [7]:
html = HTML(html = html_str)
print(html)

<HTML url='https://example.org/'>


In [81]:
question_summaries = html.find('.question-summary')
# print(summary)

In [28]:
print(question_summaries[0].text)

10296
votes
41answers
2.3m views
What does the “yield” keyword do?
What is the use of the yield keyword in Python, and what does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self....
python iterator generator yield coroutine
asked Oct 23 '08 at 22:21
Alex. S.
117k1515 gold badges4949 silver badges6161 bronze badges


In [29]:
columns = ['votes', 'vote_title', 'num_answers', 'views', 'question', 'short_desc', 'tags', 'date', 'user', 'user_details']
# question = list(question_summaries[0].text) this will not work
# print(question)
question = (question_summaries[0].text).split('\n') #converting to list
print(question)

['10296', 'votes', '41answers', '2.3m views', 'What does the “yield” keyword do?', "What is the use of the yield keyword in Python, and what does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self....", 'python iterator generator yield coroutine', "asked Oct 23 '08 at 22:21", 'Alex. S.', '117k1515 gold badges4949 silver badges6161 bronze badges']


In [30]:
len(columns) == len(question)

True

In [31]:
for column,value in zip(columns, question): #zip combines 2 lists
    print(column, value)

votes 10296
vote_title votes
num_answers 41answers
views 2.3m views
question What does the “yield” keyword do?
short_desc What is the use of the yield keyword in Python, and what does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self....
tags python iterator generator yield coroutine
date asked Oct 23 '08 at 22:21
user Alex. S.
user_details 117k1515 gold badges4949 silver badges6161 bronze badges


In [32]:
row_data = dict(zip(columns, question)) # zip with dict i.e. converting the zipped version of lists into dictionary
print(row_data)

{'votes': '10296', 'vote_title': 'votes', 'num_answers': '41answers', 'views': '2.3m views', 'question': 'What does the “yield” keyword do?', 'short_desc': "What is the use of the yield keyword in Python, and what does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self....", 'tags': 'python iterator generator yield coroutine', 'date': "asked Oct 23 '08 at 22:21", 'user': 'Alex. S.', 'user_details': '117k1515 gold badges4949 silver badges6161 bronze badges'}


In [41]:
# key_names = ['question','votes','tags']
# class_names = ['.question-hyperlink', '.votes','.tags'] # classes on stackverflow site
# the_question_element = question_summaries[0]
##print(the_question_element.find('.question-hyperlink', first = True).text) #What does the “yield” keyword do?
#print(the_question_element.find('.votes', first= True).text.replace("\nvotes", "")) #10296

In [65]:
def clean_scraped_data(sub_element, keyname= None):
    if(keyname == 'votes'):
        return sub_element.replace("\nvotes","")
    return sub_element

In [90]:
def parse_tagged_page(html):
    question_summaries = html.find('.question-summary')
    key_names = ['question','votes','tags']
    class_names = ['.question-hyperlink', '.votes','.tags'] # classes on stackverflow site
    data = []
    for question_summary in question_summaries:
        question_data = {}
        for i,_class in enumerate(class_names):
            sub_element = question_summary.find(_class, first=True)
            key_name = key_names[i]
            question_data[key_name] = clean_scraped_data(sub_element.text, keyname = key_name)
        data.append(question_data)
    return data

In [49]:
# scrape_data()
# data[0]

{'question': 'What does the “yield” keyword do?',
 'votes': '10296',
 'tags': 'python iterator generator yield coroutine'}

In [91]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200,299):
        return []
    html_str = r.text
    html = HTML(html = html_str)
    data = parse_tagged_page(html)
    return data

In [96]:
def scrape_tag(tag="python", query_filter="Votes", max_pages = 5, page_size= 10):
    base_url = "https://stackoverflow.com/questions/tagged/"
    complete_data = []
    for i in range(max_pages):
        page_num = i+1
        url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={page_size}"
        complete_data += extract_data_from_url(url)
#         time.sleep(1.2)
    return complete_data

In [97]:
datas = scrape_tag()
print(datas)

[{'question': 'What does the “yield” keyword do?', 'votes': '10296', 'tags': 'python iterator generator yield coroutine'}, {'question': 'What does if __name__ == “__main__”: do?', 'votes': '6135', 'tags': 'python namespaces main python-module idioms'}, {'question': 'Does Python have a ternary conditional operator?', 'votes': '6117', 'tags': 'python operators ternary-operator conditional-operator'}, {'question': 'What are metaclasses in Python?', 'votes': '5755', 'tags': 'python oop metaclass python-datamodel'}, {'question': 'How do I check whether a file exists without exceptions?', 'votes': '5660', 'tags': 'python file file-exists'}, {'question': 'Calling an external command from Python', 'votes': '4935', 'tags': 'python shell terminal subprocess command'}, {'question': 'How do I merge two dictionaries in a single expression in Python?', 'votes': '4879', 'tags': 'python dictionary merge'}, {'question': 'How can I safely create a nested directory?', 'votes': '4294', 'tags': 'python exc

In [98]:
len(datas)

250

In [100]:
df = pd.DataFrame(datas)
df.head(n=10)

,question,votes,tags
0,What does the “yield” keyword do?,10296,python iterator generator yield coroutine
1,What does if __name__ == “__main__”: do?,6135,python namespaces main python-module idioms
2,Does Python have a ternary conditional operator?,6117,python operators ternary-operator conditional-...
3,What are metaclasses in Python?,5755,python oop metaclass python-datamodel
4,How do I check whether a file exists without e...,5660,python file file-exists
5,Calling an external command from Python,4935,python shell terminal subprocess command
6,How do I merge two dictionaries in a single ex...,4879,python dictionary merge
7,How can I safely create a nested directory?,4294,python exception path directory operating-system
8,Accessing the index in 'for' loops?,3654,python loops list
9,Difference between staticmethod and classmethod,3622,python oop methods python-decorators


In [103]:
df.shape

(250, 3)

In [104]:
df.to_csv("python.csv", index = False)